In [ ]:
#!pip install --target=$my_path catboost

In [ ]:
# 라이브러리 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split,cross_validate, StratifiedKFold, GridSearchCV # 학습, 테스트set 구분
import sklearn.metrics as mt # 성능지표를 계산하기 위해 import
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/Dacon/train.csv')
test  = pd.read_csv('/content/drive/MyDrive/Dacon/test.csv')

In [ ]:
train = df.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)
# train = train.fillna(0) #test 결측치 0으로 채우기
# test = test.fillna(0)

In [ ]:
#train을 target과 feature로 나눠줍니다.
train_x=train.drop(['nerdiness'], axis=1)
train_y=train['nerdiness']

# Data split

In [ ]:
# 학습, 테스트 데이터 분리 (0.7:0.3)
seed = 2022
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=seed, stratify = train_y)

# 모델 생성

In [ ]:
ml_lgbm = LGBMClassifier(
            n_estimators=1000
        )

# 교차검증

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

kfold = StratifiedKFold(n_splits=5)
cv_accuracy = []
n_iter =0

for train_index, test_index in kfold.split(train_x, train_y):  # feautres 데이터를 위에서 지정한 kfold 숫자로 분할
    x_valid, x_test_fold = train_x.iloc[train_index], train_x.iloc[test_index]
    y_valid, y_test_fold = train_y[train_index], train_y[test_index]
    
    ml_lgbm.fit(x_valid, y_valid)
    pred = ml_lgbm .predict(x_test_fold)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test_fold, pred), 4) # 소수점 4자리 반올림
    train_size = x_valid.shape[0]
    test_size = x_test_fold.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

#AUC 측정(대회 지표)

In [ ]:
# ml_lgbm.fit(x_train, y_train)

# ml_lgbm_pred = ml_lgbm.predict(x_test)

In [ ]:
# # 학습결과 평가
# print("Train_Accuracy : ", ml_lgbm.score(x_train, y_train))
# print("Test_Accuracy : ", ml_lgbm.score(x_test, y_test), '\n')

# accuracy = mt.accuracy_score(y_test, ml_lgbm_pred)
# print("Accuracy :" ,mt.accuracy_score(y_test, ml_lgbm_pred))

Train_Accuracy :  1.0
Test_Accuracy :  0.7328209764918626 

Accuracy : 0.7328209764918626


전체 제공 train에 대해 split 없이 학습 진행

# Submission

In [ ]:
# 학습한 모델로 예측수행
pred_submission = ml_lgbm.predict(test)

In [ ]:
# # submission용. train 전체 학습 & test에 대해 predict 진행
# lgbm_clf = LGBMClassifier(
#             n_estimators=1000
#         )
# lgbm_clf.fit(train_x, train_y)

# lgbm_pred = lgbm_clf.predict(test)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/sample_submission.csv')

submission

,index,nerdiness
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
35447,35447,-1
35448,35448,-1
35449,35449,-1
35450,35450,-1


In [ ]:
submission["nerdiness"] = pred_submission

In [ ]:
submission

,index,nerdiness
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
35447,35447,1
35448,35448,1
35449,35449,1
35450,35450,1


In [ ]:
submission.to_csv("/content/drive/MyDrive/Dacon/baseline.csv", index = False)